# PyHEP 2022 Notebook Talk ― intro

In [ ]:
%matplotlib widget
import logging
import os

import ipywidgets
import matplotlib.pyplot as plt
import numpy as np
from ampform.io import aslatex
from IPython.display import Math

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
JAX_LOGGER = logging.getLogger("absl")
JAX_LOGGER.setLevel(logging.ERROR)


def plot_distributions():
    fig, (ax1, ax2) = plt.subplots(figsize=(7, 4.3), ncols=2)
    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False
    ax1.hist2d(*cartesian_data.values(), bins=100, cmap=plt.cm.coolwarm)
    ax2.hist2d(polar_data["phi"], polar_data["r"], bins=100, cmap=plt.cm.coolwarm)
    fig.suptitle("Hit-and-miss intensity distribution")
    ax1.set_title("cartesian")
    ax2.set_title("polar")
    ax1.set_xlabel("$x$")
    ax1.set_ylabel("$y$")
    ax2.set_xlabel(R"$\phi$")
    ax2.set_ylabel("$r$")
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax2.set_xticks([-np.pi / 2, 0, np.pi / 2])
    ax2.set_yticks([])
    ax2.set_xticklabels([r"$-\frac{\pi}{2}$", "0", r"$+\frac{\pi}{2}$"])
    fig.tight_layout()
    plt.show()


def plot_interactive():
    size = 200
    X, Y = np.meshgrid(
        np.linspace(-5, +5, size),
        np.linspace(-5, +5, size),
    )
    cartesian_domain = {"x": X, "y": Y}
    polar_domain = converter(cartesian_domain)

    fig, ax_interactive = plt.subplots(figsize=(5, 5), tight_layout=True)
    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False
    ax_interactive.set_xticks([])
    ax_interactive.set_yticks([])
    ax_interactive.set_xlabel("$x$")
    ax_interactive.set_ylabel("$y$")
    color_mesh = None

    @ipywidgets.interact(
        dphi=ipywidgets.FloatSlider(value=0, min=0, max=np.pi, step=np.pi / 100),
        k_r=(0, 3.0, np.pi / 100),
        k_phi=(0, 6),
        sigma=(0.1, 5),
    )
    def plot(dphi, k_r, k_phi, sigma):
        nonlocal color_mesh, X, Y
        polar_function.update_parameters(
            {R"\Delta\phi": dphi, "k_r": k_r, "k_phi": k_phi, "sigma": sigma}
        )
        Z = polar_function(polar_domain)
        if color_mesh is not None:
            color_mesh.remove()
        color_mesh = ax_interactive.pcolormesh(X, Y, Z, cmap=plt.cm.coolwarm)

## From mathematics to numerics

In [ ]:
import sympy as sp

x, y, a, b = sp.symbols("x y a b")
expression = x**3 + sp.sin(y / 5) ** 2
expression

In [ ]:
from tensorwaves.function.sympy import create_function

numpy_function = create_function(expression, backend="numpy")
tf_function = create_function(expression, backend="tensorflow")
jax_function = create_function(expression, backend="jax", use_cse=False)

In [ ]:
from tensorwaves.function import get_source_code

src = get_source_code(jax_function)
print(src)

In [ ]:
sample_size = 1_000_000
data = {
    "x": np.random.uniform(-50, +50, sample_size),
    "y": np.random.uniform(0.1, 2.0, sample_size),
}

In [ ]:
numpy_function(data)

In [ ]:
tf_function(data)

In [ ]:
jax_function(data)

## Data transformation

In [ ]:
r, phi, dphi, k_phi, k_r, sigma = sp.symbols(R"r phi \Delta\phi k_phi k_r sigma")
expression = (
    sp.exp(-r / sigma) * sp.sin(k_r * r) ** 2 * sp.cos(k_phi * (phi + dphi)) ** 2
)
expression

In [ ]:
from tensorwaves.function.sympy import create_parametrized_function

polar_function = create_parametrized_function(
    expression,
    parameters={dphi: 0, k_r: 0.6, k_phi: 2, sigma: 2.5},
    backend="jax",
)

In [ ]:
cartesian_to_polar = {
    r: sp.sqrt(x**2 + y**2),
    phi: sp.Piecewise((0, sp.Eq(x, 0)), (sp.atan(y / x), True)),
}

In [ ]:
Math(aslatex(cartesian_to_polar))

In [ ]:
from tensorwaves.data import SympyDataTransformer

converter = SympyDataTransformer.from_sympy(cartesian_to_polar, backend="numpy")

In [ ]:
from tensorwaves.data import (
    IntensityDistributionGenerator,
    NumpyDomainGenerator,
    NumpyUniformRNG,
)

rng = NumpyUniformRNG()
domain_generator = NumpyDomainGenerator(boundaries={"x": (-5, 5), "y": (-5, +5)})
data_generator = IntensityDistributionGenerator(
    domain_generator, polar_function, converter
)
cartesian_data = data_generator.generate(1_000_000, rng)
polar_data = converter(cartesian_data)

In [ ]:
plot_distributions()

In [ ]:
plot_interactive()